In [1]:
from pynq.overlays.base import BaseOverlay

ol = BaseOverlay('base.bit')

In [2]:
from pynq.lib import MicroblazeLibrary

lib = MicroblazeLibrary(ol.ARDUINO, ['spi', 'gpio'])

In [3]:
# RPC handle to control DACs through SPI
dac = lib.spi_open(13, 12, 11, 17)

# chip select for DAC A and DAC B - initialise to 'high' (slave disabled)
cs_a = lib.gpio_open(10)
cs_b = lib.gpio_open(2)

cs_a.set_direction(lib.GPIO_OUT)
cs_a.write(1)

cs_b.set_direction(lib.GPIO_OUT)
cs_b.write(1)

# RPC handle for switches control through GPIO - initialise to 'high' (switch open)
switches = [lib.gpio_open(3),
            lib.gpio_open(4),
            lib.gpio_open(5),
            lib.gpio_open(6),
            lib.gpio_open(7),
            lib.gpio_open(8),
            lib.gpio_open(9),
            lib.gpio_open(14),
            lib.gpio_open(15),
            lib.gpio_open(16)]

for sw in switches:
    sw.set_direction(lib.GPIO_OUT)
    sw.write(1) # low for close switch

In [4]:
LTC2662_CMD_POWER_DOWN_N = 0x40
LTC2662_CMD_SPAN = 0x60
LTC2662_CMD_WRITE_N_UPDATE_N = 0x30

# function to transfer block through SPI
def dac_write(cs_handle, dac_command, selected_dac, code):
    tx_array = bytearray(4)
    rx_array = bytearray(4)
    
    dac_code = bytearray(2)
    dac_code[:] = bytearray(int.to_bytes(code, 2, 'little'))
    
    tx_array[0] = 0 # first byte is zero
    tx_array[1] = dac_command | selected_dac # command is upper 4 bits, selected DAC is lower 4 bits
    tx_array[2] = dac_code[1]
    tx_array[3] = dac_code[0]
    
    cs_handle.write(0)
    dac.transfer(tx_array, rx_array, 4) # 4 bytes data to read
    cs_handle.write(1)
    
    fault_reg = rx_array[0]
    
    return fault_reg

In [5]:
def switch_control(sw, open_close):
    """ Opens/closes switches in the switch arrays based on 'sw' argument
        'open_close' = '1' is switch closed and '0' is switch open
    """
    if(open_close):
        switches[sw-1].write(0)
    else:
        switches[sw-1].write(1)

def dac_config(channel, current_ref):
    """ sets softspan range of all DACs based on 'current_ref'
    """
    fault_reg = 0
    
    selected_dac = (channel - 1) % 5
    
    user_command = 0
    
    if(current_ref == 0):
        user_command = 1
    elif(current_ref == 1):
        user_command = 2
    elif(current_ref == 2):
        user_command = 3
    elif(current_ref == 3):
        user_command = 4
    elif(current_ref == 4):
        user_command = 5
    elif(current_ref == 5):
        user_command = 6
        
    if(channel > 0 and channel <= 5):
        fault_reg |= dac_write(cs_a, LTC2662_CMD_SPAN, selected_dac, user_command)
    elif(channel > 5 and channel <= 10):
        fault_reg |= dac_write(cs_b, LTC2662_CMD_SPAN, selected_dac, user_command)
        
    return fault_reg
        
def dac_channel_control(channel, on_off, current_code):
    """ Switches on/off DAC channel indicated by 'channel' argument and considers both DACs
        'channel' => 1->10
        'on_off' => '1' is on and '0' is off
        'current_code' => current value in DAC counts
    """
    fault_reg = 0
    
    selected_dac = (channel - 1) % 5
    
    if(current_code == 0):
        on_off = 0
        
    dac_code = current_code
    
    if(on_off):
        if(channel > 0 and channel <= 5):
            fault_reg |= dac_write(cs_a, LTC2662_CMD_WRITE_N_UPDATE_N, selected_dac, dac_code)
        elif(channel > 5 and channel <= 10):
            fault_reg |= dac_write(cs_b, LTC2662_CMD_WRITE_N_UPDATE_N, selected_dac, dac_code)            
    else:
        if(channel > 0 and channel <= 5):
            fault_reg |= dac_write(cs_a, LTC2662_CMD_POWER_DOWN_N, selected_dac, 0)
        elif(channel > 5 and channel <= 10):
            fault_reg |= dac_write(cs_b, LTC2662_CMD_POWER_DOWN_N, selected_dac, 0)
            
    return fault_reg

# testing

In [12]:
# SWITCH OFF ALL DAC CHANNELS
for i in range(10):
    dac_channel_control(i+1, 0, 0)

In [7]:
# SET SOFTSPAN RANGE FOR ALL DACS
for i in range(10):
    dac_config(i+1, 5)

In [8]:
# # ALL SWITCHES CLOSE
# for i in range(10):
#     switch_control(i+1, 1)

In [9]:
# # ALL LEDS ON
# for i in range(10):
#     dac_channel_control(i+1, 1, 16384)

# dashboard test

In [10]:
import ipywidgets as widgets

hbox = widgets.Box(layout=widgets.Layout(display='inline-flex', flex_flow='row nowrap', justify_content='space-between'))

vbox1 = widgets.Box(layout=widgets.Layout(display='inline-flex', flex_flow='column nowrap', justify_content='flex-start', border='solid 1px blue'))

channel_select = widgets.Dropdown(options=range(1,11), layout={'width' : '100px'})
switch_select = widgets.Dropdown(options=range(1,11))

vbox1.children += (widgets.Label(value='select channel:'), channel_select, widgets.Label(value='select switch:'), switch_select,)

hbox.children += (vbox1,)

vbox2 = widgets.Box(layout=widgets.Layout(display='inline-flex', flex_flow='column nowrap', justify_content='flex-start', border='solid 1px blue'))

brightness_slider = widgets.IntSlider(desription='dac counts: ', value=8192, min=0, max=65535)
led_on_button = widgets.Button(description='LED on')
led_off_button = widgets.Button(description='LED off')
switch_open_button = widgets.Button(description='open switch')
switch_close_button = widgets.Button(description='close switch')

vbox2.children += (brightness_slider, led_on_button, led_off_button, switch_open_button, switch_close_button)

import functools

def dac_channel_control_wrapper(channel_select, on_off, brightness_slider, btn):
    dac_channel_control(channel_select.value, on_off, brightness_slider.value)
    
def switch_control_wrapper(switch_select, open_close, btn):
    switch_control(switch_select.value, open_close)

led_on_button.on_click(functools.partial(dac_channel_control_wrapper, channel_select, 1, brightness_slider))
led_off_button.on_click(functools.partial(dac_channel_control_wrapper, channel_select, 0, brightness_slider))
switch_open_button.on_click(functools.partial(switch_control_wrapper, switch_select, 0))
switch_close_button.on_click(functools.partial(switch_control_wrapper, switch_select, 1))

hbox.children += (vbox2,)

In [11]:
hbox

Box(children=(Box(children=(Label(value='select channel:'), Dropdown(layout=Layout(width='100px'), options=(1,…

In [10]:
vbox1

Box(children=(Label(value='select channel:'), Dropdown(layout=Layout(width='100px'), options=(1, 2, 3, 4, 5, 6…

In [11]:
vbox2

Box(children=(IntSlider(value=8192, max=65535), Button(description='LED on', style=ButtonStyle()), Button(desc…

In [16]:
widgets.Button()

Button(style=ButtonStyle())

# hearlight